In [ ]:
!pip install planet aiohttp rasterio shapely fiona geopandas numpy

In [ ]:
import os
import time
from pathlib import Path
import numpy as np
import geopandas as gpd
from shapely.geometry import mapping
import rasterio
import requests
import planet
from planet import Planet, data_filter, order_request, Session, OrdersClient, reporting
import asyncio
from datetime import datetime
from shapely.geometry import Polygon, MultiPolygon
from shapely.ops import unary_union

# Setup

In [ ]:
PLANET_API_KEY = "YOUR-API-KEY"
SHAPEFILE_PATH = "YOUR-SHAPEFILE-PATH"

In [ ]:
DATE_START = datetime(2026, 1, 1)
DATE_END   = datetime(2026, 1, 1)  

In [ ]:
PRODUCT_BUNDLE = "analytic_8b_sr_udm2"

# Code Execution
## Authentication and global variables

In [ ]:
os.environ["PL_API_KEY"] = PLANET_API_KEY
os.environ["PLANET_AUTH_TYPE"] = "api-key"

In [ ]:
session = requests.Session()

In [ ]:
session.auth = ('', PLANET_API_KEY)
headers = {'content-type': 'application/json'}
orders_url = 'https://api.planet.com/compute/ops/orders/v2'
PRODUCTS_URL = "https://api.planet.com/subscriptions/v1/products"

## Helper function that searches which product id you have for a given AOI

In [ ]:
async def search_item_ids_by_aoi(aoi, item_types=None, max_items=5):
    if item_types is None:
        item_types = ["PSScene"]

    async with Session() as sess:
        data_client = sess.client('data')

        geom_filter = data_filter.geometry_filter(aoi)
        date_filter = data_filter.date_range_filter(
            field_name="acquired",
            gte=DATE_START,
            lte=DATE_END
        )

        sfilter = data_filter.and_filter([geom_filter, date_filter])

        print(f"🔍 Searching scenes: {DATE_START} e {DATE_END}...")
        items_async = data_client.search(item_types, sfilter)

        items = [it async for it in items_async]
        print(f"✅ Found {len(items)} itens for AOI + data")

        item_ids = [it["id"] for it in items[:max_items]]
        print("🛰 Selected item_ids:", item_ids)

        return item_ids

## Helper function that waits for a order and downloads it

In [ ]:
async def poll_and_download(order):
    async with Session() as sess:
        cl = OrdersClient(sess)
        # Use "reporting" to manage polling for order status
        with reporting.StateBar(state='creating') as bar:
            # Grab the order ID
            bar.update(state='created', order_id=order['id'])
            # poll...poll...poll...
            await cl.wait(order['id'], callback=bar.update_state)
        # if we get here that means the order completed. Yay! Download the files.
        filenames = await cl.download_order(order['id'])
        return filenames

## Helper function that checks your API quota

In [ ]:
import requests
import pandas as pd

PRODUCTS_URL = "https://api.planet.com/account/v1/my/products"

def fetch_my_products_quota(api_key: str) -> pd.DataFrame:
    r = requests.get(
        MY_PRODUCTS_URL,
        headers={
            "Authorization": f"api-key {api_key}",
            "Content-Type": "application/json",
        },
        timeout=60
    )
    r.raise_for_status()
    data = r.json()

    rows = []
    for p in data.get("results", data): 
        rows.append({
            "product_id": p.get("id"),
            "name": p.get("name"),
            "title": p.get("title"),
            "family": p.get("product_family"),
            "supports_reservation": p.get("supports_reservation"),
            "quota_total": p.get("quota_total"),
            "quota_used": p.get("quota_used"),
            "quota_units": p.get("quota_units"),
            "unlimited_quota": p.get("unlimited_quota"),
            "term_start_date": p.get("term_start_date"),
            "term_end_date": p.get("term_end_date"),
            "resource_ids": ", ".join(p.get("resource_ids", [])),
        })

    df = pd.DataFrame(rows)
    if not df.empty and "quota_total" in df.columns and "quota_used" in df.columns:
        df["quota_remaining"] = df["quota_total"] - df["quota_used"]
    return df.sort_values(["family","name"], na_position="last").reset_index(drop=True)



## Helper function that creates an order

In [ ]:
async def main_clip_and_download(request_clip):
    # 1) create an order
    async with Session() as sess:
        orders_client = sess.client('orders')
        order = await orders_client.create_order(request_clip)
        print("🧾 Order created:", order["id"])

    # 2) wait and download
    files = await poll_and_download(order)
    print("📂 files downloaded:")
    for f in files:
        print("   -", f)

    return files


## Helper function to read a SHAPEFILE and transform to an AOI

In [ ]:
def shapefile_to_clip_aoi(shp_path: str):
    gdf = gpd.read_file(shp_path)

    if gdf.empty:
        raise ValueError("Empty Shapefile.")

    # forces CRS in WGS84 (lon/lat)
    if gdf.crs is None:
        raise ValueError("Shapefile with CRS. Set it gdf.set_crs(...).")
    gdf = gdf.to_crs(epsg=4326)

    # unify in one single polygon
    geom = unary_union(gdf.geometry)

    # if MultiPolygon, get the bigger one or adapt
    if isinstance(geom, MultiPolygon):
        geom = max(list(geom.geoms), key=lambda p: p.area)

    if not isinstance(geom, Polygon):
        raise TypeError(f"Not a Polygon/MultiPolygon: {geom.geom_type}")

    # GeoJSON-like dict 
    geo = gpd.GeoSeries([geom], crs="EPSG:4326").__geo_interface__["features"][0]["geometry"]
    # geo já sai como {"type":"Polygon","coordinates":[...]}
    return geo

clip_aoi = shapefile_to_clip_aoi(SHAPEFILE_PATH)
clip_aoi

## Checks your API quota

In [ ]:
df_quota = fetch_my_products_quota(PLANET_API_KEY)
df_quota

## Get AOI and tries to find scenes/item/ids

In [ ]:
ids = await search_item_ids_by_aoi(clip_aoi, max_items=120)
ids

### Bundles that enable the of NDVI and NDRE
- **NDVI pre-req**: Red & NIR bands (available in analytic_* bundles).
- **NDRE pre-req**: Red Edge & NIR bands (available *only* in analytic_8b_* bundles with SuperDove scenes, with 8 bands).
- ALL bundles include multispectral TIFs and GeoTIFF) with metadada for processing.

| Bundle | Description | Bands | NDVI support | NDRE support | Notes |
|--------|-----------|------------------|----------------|----------------|-------|
| **analytic_udm2** | TOA (Top of Atmosphere) Radiance with UDM2 mask (clouds, shadows, etc). | Blue, Green, Red, NIR (4 bands). | Yes (Red + NIR). | No (no Red Edge). | Good for basic NDVI. Use with old PlanetScope scenes (not with SuperDove). |
| **analytic_sr_udm2** | Atmospherically corrected surface reflectance (SR), with UDM2. | Blue, Green, Red, NIR (4 bands). | Yes (Red + NIR). | No (no Red Edge). | Better for precise analysis, once it corrects atmosphere. |
| **analytic_8b_udm2** | TOA Radiance with UDM2, for SuperDove scenes. | Coastal Blue, Blue, Green I, Green II, Yellow, Red, Red Edge, NIR (8 bands). | Yes (Red + NIR). | Yes (Red Edge + NIR). | Perfect for advanced NDRE + NDVI. |
| **analytic_8b_sr_udm2** | Atmospherically corrected surface reflectance (SR), with UDM2, for SuperDove scenes. | Coastal Blue, Blue, Green I, Green II, Yellow, Red, Red Edge, NIR (8 bands). | Yes (Red + NIR). | Yes (Red Edge + NIR). | Recommended for precision; corrects atmosphere and includes all bands. |
| **visual** | RGB corrected for vision (not for analytics). | Red, Green, Blue (3 bands). | No (no NIR). | No (no Red Edge/NIR). | Only for visualization; not for indexes. |


## Defines which bundle you want

In [ ]:
#PRODUCT_BUNDLE = "analytic_8b_sr_udm2"   # "analytic_udm2" "analytic_8b_udm2" "analytic_sr_udm2" "analytic_8b_sr_udm2" "visual"
single_product = [
    {
        "item_ids": ids,
        "item_type": "PSScene",#"SkySatCollect",
        "product_bundle": PRODUCT_BUNDLE
    }
]

clip_tool = {
    "clip": {
        "aoi": clip_aoi
    }
}

request_clip = {
  "name": " AOI clip",
  "products": single_product,
  "tools": [clip_tool]
}

In [ ]:
from pathlib import Path

def find_planet_udm2_mask(analytic_tif_path: str) -> str:
    """
    Dado o path de um arquivo Planet AnalyticMS(_SR)_8b_clip.tif,
    retorna o path correspondente do arquivo UDM2 (máscara).

    Exemplo:
    input : ..._3B_AnalyticMS_SR_8b_clip.tif
    output: ..._3B_udm2_clip.tif
    """
    p = Path(analytic_tif_path)

    name = p.name

    if "AnalyticMS" not in name:
        raise ValueError("Arquivo não parece ser um AnalyticMS da Planet")

    udm2_name = (
        name
        .replace("_AnalyticMS_SR_8b_clip.tif", "_udm2_clip.tif")
        .replace("_AnalyticMS_8b_clip.tif", "_udm2_clip.tif")
    )

    return str(p.with_name(udm2_name))

## Helper function that apply a mask into an array (used to generate NDVI and NDRE)

In [ ]:
import rasterio
import numpy as np

def apply_mask(arr: np.ndarray, valid_mask: np.ndarray) -> np.ndarray:
    out = arr.astype("float32", copy=True)
    out[~valid_mask] = np.nan
    return out

## Helper function that keeps the index matrix values in a correct range (used to generate NDVI and NDRE)

In [ ]:
def safe_index(numerator: np.ndarray, denominator: np.ndarray) -> np.ndarray:
    out = np.full_like(numerator, np.nan, dtype="float32")
    denom = denominator.astype("float32")
    num = numerator.astype("float32")
    m = np.isfinite(num) & np.isfinite(denom) & (np.abs(denom) > 1e-6)
    out[m] = num[m] / denom[m]
    return out

## Helper function that writes a new tif file for NDVI or NDRE

In [ ]:
def write_geotiff(out_path: str, template_src, array2d: np.ndarray, nodata_val: float = -9999.0):
    profile = template_src.profile.copy()
    profile.update(
        dtype="float32",
        count=1,
        nodata=nodata_val,
        compress="deflate",
        predictor=2
    )

    data = array2d.copy()
    data = np.where(np.isfinite(data), data, nodata_val).astype("float32")

    with rasterio.open(out_path, "w", **profile) as dst:
        dst.write(data, 1)


## Helper function that checks udm2 types/bands, to help with build_valid_mask_from_udm2()

In [ ]:
import rasterio
import numpy as np

def inspect_udm2(udm2_path: str, sample=2000):
    with rasterio.open(udm2_path) as src:
        print("UDM2 bands:", src.count)
        print("UDM2 dtype:", src.dtypes)
        b1 = src.read(1).ravel()
        b1 = b1[~np.isnan(b1)] if np.issubdtype(b1.dtype, np.floating) else b1
        if len(b1) > sample:
            b1 = np.random.choice(b1, size=sample, replace=False)
        vals, counts = np.unique(b1, return_counts=True)
        top = sorted(zip(vals.tolist(), counts.tolist()), key=lambda x: x[1], reverse=True)[:20]
        print("Top values (value, count):", top)


## Function that generates NDVI index

In [ ]:
from pathlib import Path

def process_ndvi(analytic_tif_path: str, udm2_mode: str = "nonzero") -> str:
    analytic_tif_path = str(analytic_tif_path)
    udm2_path = find_planet_udm2_mask(analytic_tif_path)

    # valid mask
    valid_mask = build_valid_mask_from_udm2(udm2_path, mode=udm2_mode)

    out_path = str(Path(analytic_tif_path).with_name(Path(analytic_tif_path).stem + "_NDVI.tif"))

    with rasterio.open(analytic_tif_path) as src:
        # red = band 6, nir = band 8
        red = src.read(6).astype("float32")
        nir = src.read(8).astype("float32")

        red = apply_mask(red, valid_mask)
        nir = apply_mask(nir, valid_mask)

        ndvi = safe_index(nir - red, nir + red)

        write_geotiff(out_path, src, ndvi)

    return out_path


## Function that generates NDRE index

In [ ]:
def process_ndre(analytic_tif_path: str, udm2_mode: str = "nonzero") -> str:
    analytic_tif_path = str(analytic_tif_path)
    udm2_path = find_planet_udm2_mask(analytic_tif_path)

    valid_mask = build_valid_mask_from_udm2(udm2_path, mode=udm2_mode)

    out_path = str(Path(analytic_tif_path).with_name(Path(analytic_tif_path).stem + "_NDRE.tif"))

    with rasterio.open(analytic_tif_path) as src:
        rededge = src.read(7).astype("float32")
        nir = src.read(8).astype("float32")

        rededge = apply_mask(rededge, valid_mask)
        nir = apply_mask(nir, valid_mask)

        ndre = safe_index(nir - rededge, nir + rededge)

        write_geotiff(out_path, src, ndre)

    return out_path

## Helper function that builds a valid mask to be used in indexes processing, to avoid clouds, snow, shadows, etc.

In [ ]:
def build_valid_mask_from_udm2(udm2_path: str, mode: str = "nonzero") -> np.ndarray:
    """
    Returns a boolean mask (True=valid) from UDM2.

    mode:
      - "nonzero": valid = (band1 != 0)
      - "zero_is_valid": valid = (band1 == 0)
      - "one_is_valid": valid = (band1 == 1)

    Use inspect_udm2() to decide the correct UDM2 mode.
    """
    with rasterio.open(udm2_path) as src:
        b1 = src.read(1)

    if mode == "nonzero":
        valid = (b1 != 0)
    elif mode == "zero_is_valid":
        valid = (b1 == 0)
    elif mode == "one_is_valid":
        valid = (b1 == 1)
    else:
        raise ValueError("invalid mode. Use: nonzero | zero_is_valid | one_is_valid")

    return valid


## Helper function that reads the GeoTIFF with 1 band and returns (array2d float, nodata). For nodata converts to NaN.

In [ ]:
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import seaborn as sns

def _read_index_raster(path: str):
    with rasterio.open(path) as src:
        arr = src.read(1).astype("float32")
        nodata = src.nodata

    if nodata is not None:
        arr[arr == nodata] = np.nan

    # Also removes inf/-inf 
    arr[~np.isfinite(arr)] = np.nan
    return arr, nodata

## Helper function that normalize the pixels distribution and generates Distribution/Density charts.

In [ ]:
def _show_index(path: str, title: str, cmap: str, clip_range: tuple[float, float]):
    arr, nodata = _read_index_raster(path)

    # Clipping
    vmin, vmax = clip_range
    arr_clip = np.clip(arr, vmin, vmax)

    # Flatten
    values = arr_clip[~np.isnan(arr_clip)].ravel()

    if values.size == 0:
        raise ValueError(f"Raster with no valid pixels after remove nodata/NaN: {path}")

    # --- FIG 1 ---
    plt.figure(figsize=(10, 6))
    im = plt.imshow(arr_clip, cmap=cmap, vmin=vmin, vmax=vmax)
    plt.title(title)
    plt.axis("off")
    cbar = plt.colorbar(im, fraction=0.046, pad=0.04)
    cbar.set_label("Index value")
    plt.show()

    # --- FIG 2 ---
    sns.set_theme(style="whitegrid")
    plt.figure(figsize=(10, 4.5))
    ax = sns.histplot(values, bins=60, kde=True, stat="density")
    ax.set_title(f"Distribution — {title}")
    ax.set_xlabel("Index value (clipped)")
    ax.set_ylabel("Density")

    # Ref lines
    ax.axvline(np.nanmean(values), linestyle="--", linewidth=1.5, label=f"Mean: {np.nanmean(values):.3f}")
    ax.axvline(np.nanmedian(values), linestyle=":", linewidth=2.0, label=f"Median: {np.nanmedian(values):.3f}")
    ax.legend()
    plt.show()

## Function that shows NDVI geo visualization + Distribution/Density charts

In [ ]:
def show_ndvi(path: str):
    _show_index(
        path=path,
        title="NDVI",
        cmap="RdYlGn",          
        clip_range=(-1.0, 1.0)
    )



## Function that shows NDRE geo visualization + Distribution/Density charts

In [ ]:
def show_ndre(path: str):
    _show_index(
        path=path,
        title="NDRE",
        cmap="viridis",         
        clip_range=(-0.2, 0.8)
    )

## Helper function that checks tif bands

In [ ]:
def is_ndvi_ndre_supported(path):
    result = None
    with rasterio.open(path) as src:
        if src.count > 3:
            if 'red' in src.descriptions and 'blue' in src.descriptions and 'green' in src.descriptions and 'nir' in src.descriptions:
                result = 'ndvi'
            if 'red' in src.descriptions and 'blue' in src.descriptions and 'green' in src.descriptions and 'nir' in src.descriptions and 'rededge' in src.descriptions:
                result = 'ndvi_ndre'
    return result                

## Create order, process it, wait and download

In [ ]:
files = await main_clip_and_download(request_clip)
files

## Generates NDVI and NDRE data and files

In [ ]:
for f in files:
    fname = f.as_posix()
    if '.tif' and '_SR_' in fname:
        indexes_available = is_ndvi_ndre_supported(fname)
        if 'ndvi' in indexes_available: 
            ndvi = process_ndvi(fname)
            print(ndvi)
            show_ndvi(ndvi)
        if 'ndre' in indexes_available: 
            ndre = process_ndre(fname)
            print(ndre)
            show_ndre(ndre)
